In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
sys.path.append("/home/lucas/Attention-Is-All-You-Need/")

from transformer import TransformerFromScratch

In [2]:
transfomer_from_scratch = TransformerFromScratch(3,4,4,120,512,64,64,4,2048)

In [3]:
input_tensor = torch.randn(2,3,512)
tgt_tensor = torch.randn(2,3,512)

mask = torch.triu(torch.ones((3,3),dtype=torch.int8)).T

In [4]:
out = transfomer_from_scratch(input_tensor,tgt_tensor,tgt_mask=mask)

In [5]:
out.shape

torch.Size([2, 3, 120])

In [6]:
out.argmax(dim=-2)

tensor([[0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
         2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
         2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0,
         0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
        [0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2,
         0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 1, 0,
         0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2,
         2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
         0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2]])

In [2]:
encoder = Encoder(3,4,512,64,64,4,2048)
decoder = Decoder(3,4,512,64,64,4,2048)

In [4]:
mask

tensor([[1, 0, 0],
        [1, 1, 0],
        [1, 1, 1]], dtype=torch.int8)

In [5]:
x = encoder(input_tensor)

In [6]:
output = decoder(tgt_tensor, x, tgt_mask=mask)

torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])


In [7]:
output = decoder(tgt_tensor, x)

In [17]:
output

tensor([[[-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720],
         [-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720],
         [-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720]],

        [[ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986],
         [ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986],
         [ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986]]],
       grad_fn=<NativeLayerNormBackward0>)

In [19]:
output

tensor([[[-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720],
         [-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720],
         [-0.5692, -0.3596,  0.3554,  ..., -0.3345, -0.9155,  0.1720]],

        [[ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986],
         [ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986],
         [ 0.0579, -1.1439,  1.4661,  ..., -0.5045, -0.1635, -1.2986]]],
       grad_fn=<NativeLayerNormBackward0>)

In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, seq_length:int, d_model:int, n = 10000, dtype = torch.float32):
        super(PositionalEncoding,self).__init__()
        self.seq_length = seq_length
        self.d_model = d_model
        self.n = n
        self.dtype = dtype
        self.encode_table = self._create_table()

    def _create_table(self) -> torch.tensor:
        table = torch.zeros((self.seq_length, self.d_model),dtype=self.dtype)

        for pos in torch.arange(self.seq_length):
            # Here d_model is divide by 2 because the 2*i and 2*i + 1
            # The interaction foward two by two 
            for i in torch.arange(self.d_model//2):
                denominator = 2*i/self.d_model
                calculation = pos/torch.pow(self.n, denominator)
                table[pos,2*i]      = torch.sin(calculation) 
                table[pos,2*i + 1]  = torch.cos(calculation)

        return table

    def forward(self,x:torch.tensor) -> torch.tensor:
        x += self.encode_table

        return x

In [4]:
def scaled_dot_product_attention(query:torch.tensor, key:torch.tensor, value:torch.tensor, mask=None) -> [torch.tensor,torch.tensor]:
    factor = 1/torch.sqrt(torch.tensor(key.size(-1)))
    attn = torch.matmul(query,key.transpose(-2,-1))*factor
    if(mask is not None):
        attn = attn.masked_fill(mask == 0, -torch.inf)
    attn = F.softmax(attn,dim=-1)
    
    x = torch.matmul(attn,value)

    return x, attn

In [5]:
class HeadAttention(nn.Module):
    def __init__(self, d_model:int, d_k:int, d_v:int):
        super(HeadAttention,self).__init__()
        self.weights_query = nn.Parameter(torch.randn(d_model,d_k))
        self.weights_key = nn.Parameter(torch.randn(d_model,d_k))
        self.weights_value = nn.Parameter(torch.randn(d_model,d_v))

    def forward(self, query:torch.tensor, key:torch.tensor, value:torch.tensor, mask=None) -> torch.tensor:
        q = torch.matmul(query,self.weights_query)
        k = torch.matmul(key,self.weights_key)
        v = torch.matmul(value,self.weights_value)

        x, _ = scaled_dot_product_attention(q,k,v,mask=mask)
        return x
        

In [6]:
class MultiHead(nn.Module):
    def __init__(self, d_model:int, d_k:int, d_v:int, h:int):
        super(MultiHead,self).__init__()
        self.weights_concat = nn.Parameter(torch.randn(d_v*h,d_model))

        self.multi_head = nn.ModuleList([
            HeadAttention(d_model=d_model, d_k=d_k, d_v=d_v) for _ in range(h)
        ])

        self.norm_layer = nn.LayerNorm(d_model)

    def forward(self, query:torch.tensor, key:torch.tensor, value:torch.tensor, mask=None) -> torch.tensor:
        x = torch.concat([module_(query, key, value, mask=mask) for module_ in self.multi_head],dim=-1)
        x = torch.matmul(x,self.weights_concat)

        x += query

        x = self.norm_layer(x)
        
        return x

In [7]:
class PositionWiseFeedForwardNetworks(nn.Module):
    def __init__(self, d_model:int, d_ff:int):
        super(PositionWiseFeedForwardNetworks,self).__init__()
        self.linear_inner_layer = nn.Linear(d_model,d_ff)
        self.linear_layer = nn.Linear(d_ff,d_model)

        self.norm_layer = nn.LayerNorm(d_model)

    def forward(self, x:torch.tensor) -> torch.tensor:
        x = self.linear_inner_layer(x)
        x = x.relu()
        x = self.linear_layer(x)

        x += x
        x = self.norm_layer(x)

        return x

In [8]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model:int, d_k:int, d_v:int, h:int, d_ff:int):
        super(EncoderLayer,self).__init__()
        self.multi_head_attention = MultiHead(d_model=d_model,d_k=d_k,d_v=d_v,h=h)
        self.position_wise_feed_forward_networks = PositionWiseFeedForwardNetworks(d_model=d_model,d_ff=d_ff)

    def forward(self, query:torch.tensor, key:torch.tensor, value:torch.tensor, mask=None) -> torch.tensor:
        x = self.multi_head_attention(query=value, key=key, value=value, mask=mask)
        x = self.position_wise_feed_forward_networks(x)
        return x

In [9]:
class Encoder(nn.Module):
    def __init__(self, seq_length: int, n_layers:int, d_model:int, d_k:int, d_v:int, h:int, d_ff:int):
        super(Encoder,self).__init__()
        self.positional_encoding = PositionalEncoding(seq_length=seq_length,d_model=d_model)
        self.encoder_layer = nn.ModuleList([
            EncoderLayer(d_model=d_model,d_k=d_k,d_v=d_v,h=h,d_ff=d_ff) for _ in range(n_layers)
        ])

    def forward(self, src:torch.tensor, mask=None) -> torch.tensor:
        src = self.positional_encoding(src)
        for module_ in self.encoder_layer:
            src += module_(src,src,src,mask)

        return src
        

In [10]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model:int, d_k:int, d_v:int, h:int, d_ff:int):
        super(DecoderLayer,self).__init__()
        self.multi_head_attention = MultiHead(d_model=d_model,d_k=d_k,d_v=d_v,h=h)
        self.multi_head_attention_memory = MultiHead(d_model=d_model,d_k=d_k,d_v=d_v,h=h)
        self.position_wise_feed_forward_networks = PositionWiseFeedForwardNetworks(d_model=d_model,d_ff=d_ff)

    def forward(self, query:torch.tensor, key:torch.tensor, value:torch.tensor, memory:torch.tensor, mask=None) -> torch.tensor:
        x = self.multi_head_attention(query=value, key=key, value=value, mask=mask)
        x = self.multi_head_attention_memory(query=x, key=memory, value=memory)
        x = self.position_wise_feed_forward_networks(x)

        return x

In [14]:
class Decoder(nn.Module):
    def __init__(self, seq_length: int, n_layers:int, d_model:int, d_k:int, d_v:int, h:int, d_ff:int):
        super(Decoder,self).__init__()
        self.positional_encoding = PositionalEncoding(seq_length=seq_length,d_model=d_model)
        self.encoder_layer = nn.ModuleList([
            DecoderLayer(d_model=d_model,d_k=d_k,d_v=d_v,h=h,d_ff=d_ff) for _ in range(n_layers)
        ])

    def forward(self, src:torch.tensor, memory:torch.tensor, mask=None) -> torch.tensor:
        src = self.positional_encoding(src)
        for module_ in self.encoder_layer:
            src += module_(src,src,src,memory,mask)

        return src
        

In [2]:
encoder = Encoder(3,4,512,64,64,4,2048)

In [3]:
decoder = Decoder(3,4,512,64,64,4,2048)

In [4]:
input_tensor = torch.randn(2,3,512)
tgt_tensor = torch.randn(2,3,512)

mask = torch.triu(torch.ones((3,3),dtype=torch.int8)).T

In [5]:
x = encoder(input_tensor)

In [7]:
x.shape

torch.Size([2, 3, 512])

In [11]:
decoder(tgt_tensor,tgt_tensor)

tensor([[[-1.6582, -0.8472,  1.2657,  ...,  0.1043,  1.1291, -0.4406],
         [-1.6582, -0.8472,  1.2657,  ...,  0.1043,  1.1291, -0.4406],
         [-1.6582, -0.8472,  1.2657,  ...,  0.1043,  1.1291, -0.4406]],

        [[-1.3110, -1.6264,  0.0674,  ...,  0.9752,  1.7127,  0.5505],
         [-1.3110, -1.6264,  0.0674,  ...,  0.9752,  1.7127,  0.5505],
         [-1.3110, -1.6264,  0.0674,  ...,  0.9752,  1.7127,  0.5505]]],
       grad_fn=<NativeLayerNormBackward0>)

In [79]:
mask = torch.triu(torch.ones((3,3),dtype=torch.int8)).T

In [80]:
encoder(input_tensor,mask)

tensor([[[ 1.0219, -1.3973, -0.3051,  ...,  2.0317, -0.6028,  5.2137],
         [ 3.6245, -0.2907,  1.7433,  ...,  0.3274,  0.8121,  7.7216],
         [-1.1843, -1.7584,  1.5053,  ...,  1.3223,  2.6765,  6.5351]],

        [[-2.4180, -4.9422,  3.7338,  ...,  3.6649,  1.1594,  4.1164],
         [-0.7324, -3.0181,  2.3415,  ...,  1.3145,  2.9303,  3.7378],
         [ 1.0661, -1.7807,  4.3706,  ..., -0.2056, -0.0184,  4.5398]]],
       grad_fn=<AddBackward0>)